Importing and Cleaning Franklin's Data

In [2]:
# Packages
# For Cleaning and basic functions
import pandas as pd
import numpy as np
import requests
import json

# Connecting to Spotify API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [12]:
# Load the Excel file into a pandas DataFrame
franklin = "https://raw.githubusercontent.com/lphan1812/spotify-project/main/franklins.csv"

data = pd.read_csv(franklin)

#Delete the first row
data = data.drop(index=0)

print(data.columns)


Index(['Unnamed: 0', 'ts', 'username', 'platform', 'ms_played', 'conn_country',
       'ip_addr_decrypted', 'user_agent_decrypted',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name',
       'episode_show_name', 'spotify_episode_uri', 'reason_start',
       'reason_end', 'shuffle', 'skipped', 'offline', 'offline_timestamp',
       'incognito_mode'],
      dtype='object')


In [13]:
# Load the Excel file into a pandas DataFrame
#data = pd.read_excel("franklin_data.xlsx", header=1)

# Split the spotify_track_uri column into three columns
data[['spotify', 'track', 'track_uri']] = data['spotify_track_uri'].str.split(':', n=2, expand=True)

# Remove the 'spotify:track:' string from the track_uri column
data['track_uri'] = data['track_uri'].str.replace('spotify:track:', '')

# Drop the 'spotify' and 'track' columns since we don't need them
data = data.drop(columns=['spotify', 'track'])

# Print the first few rows to verify the results
data.head()


,Unnamed: 0,ts,username,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,...,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,track_uri
1,1,2018-01-23T13:58:46Z,sir_frankster,"iOS 11.1.1 (iPhone8,1)",237300,US,66.87.31.235,unknown,7 Years,Lukas Graham,...,NaN,NaN,trackdone,trackdone,True,NaN,False,1.516720e+12,False,5kqIPrATaCc2LqxVWzQGbk
2,2,2021-10-20T18:41:27Z,sir_frankster,"iOS 12.0.1 (iPhone6,1)",242013,US,107.127.45.51,unknown,drivers license,Olivia Rodrigo,...,NaN,NaN,trackdone,trackdone,False,NaN,False,1.634760e+12,False,5wANPM4fQCJwkGd4rN57mH
3,3,2020-12-01T05:25:31Z,sir_frankster,Windows 10 (10.0.18363; x64; AppX),81544,US,108.53.217.176,unknown,Piano Man,Billy Joel,...,NaN,NaN,trackdone,logout,False,NaN,False,1.606600e+12,False,78WVLOP9pN0G3gRLFy1rAa
4,4,2019-05-25T04:59:44Z,sir_frankster,OS X 10.14.4 [x86 8],2316,US,100.8.116.86,unknown,Push It to the Limit,Corbin Bleu,...,NaN,NaN,clickrow,endplay,True,NaN,False,1.558760e+12,False,3K01q0jtSNHeqx5gs97HH2
5,5,2022-12-12T04:28:57Z,sir_frankster,osx,179491,US,24.2.197.91,unknown,Here You Come Again,Dolly Parton,...,NaN,NaN,trackdone,trackdone,False,0.0,False,1.670819e+09,False,6H7nDglS6xWpRidbhMwI2L


In [14]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = '08923f56c32e46c1ae4b4ff05260b76f'
CLIENT_SECRET = '226fd5b07d874db0a61e720c750d646c'

# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [15]:
data = data[data['track_uri'].notna()]
data['ts'] = pd.to_datetime(data['ts'])
last_six_months = data['ts'] >= (data['ts'].max() - pd.DateOffset(months=6))
last_six_months = data[last_six_months].sort_values('ms_played', ascending=False)
last_six_months

,Unnamed: 0,ts,username,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,...,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,track_uri
22030,22030,2023-04-18 01:56:21+00:00,sir_frankster,osx,647283,US,24.2.197.91,unknown,The One That Got Away - Acoustic,Katy Perry,...,NaN,NaN,trackdone,endplay,False,1.0,False,1.681782e+09,False,4FFSK1xgCLkTZJLUERLZKG
18140,18140,2022-10-23 19:18:47+00:00,sir_frankster,ios,613026,US,166.196.106.67,unknown,All Too Well (10 Minute Version) (Taylor's Ver...,Taylor Swift,...,NaN,NaN,fwdbtn,trackdone,True,0.0,False,1.666552e+09,False,5enxwA8aAbwZbf5qCHORXi
10038,10038,2023-03-17 22:45:19+00:00,sir_frankster,ios,613026,US,107.115.20.162,unknown,All Too Well (10 Minute Version) (Taylor's Ver...,Taylor Swift,...,NaN,NaN,trackdone,trackdone,True,0.0,False,1.679093e+09,False,5enxwA8aAbwZbf5qCHORXi
906,906,2023-01-04 02:56:30+00:00,sir_frankster,osx,613026,US,24.2.197.91,unknown,All Too Well (10 Minute Version) (Taylor's Ver...,Taylor Swift,...,NaN,NaN,trackdone,trackdone,False,0.0,False,1.672800e+09,False,5enxwA8aAbwZbf5qCHORXi
9589,9589,2023-02-23 01:13:09+00:00,sir_frankster,osx,525767,US,24.2.197.91,unknown,A Million Dreams (Instrumental Guitar),Guus Dielissen,...,NaN,NaN,trackdone,trackdone,False,0.0,False,1.677114e+09,False,0T482F5L30d7ChNbLqsKa7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,2382,2022-11-22 22:27:06+00:00,sir_frankster,ios,0,US,107.122.192.162,unknown,I Want To Hold Your Hand - Remastered 2015,The Beatles,...,NaN,NaN,trackdone,endplay,True,1.0,False,1.669156e+09,False,4pbG9SUmWIvsROVLF0zF9s
4486,4486,2023-03-22 02:52:33+00:00,sir_frankster,osx,0,US,24.2.197.91,unknown,Lead Me Out of the Dark,Crown The Empire,...,NaN,NaN,trackdone,endplay,False,1.0,False,1.679454e+09,False,6h2YXTAX6QN83w0gOTjDZa
17021,17021,2023-03-07 01:38:06+00:00,sir_frankster,osx,0,US,24.2.197.91,unknown,All About Us,He Is We,...,NaN,NaN,trackdone,endplay,False,1.0,False,1.678153e+09,False,0UQAkAVNk3tubJYlOHyG3q
13426,13426,2023-03-17 21:34:15+00:00,sir_frankster,ios,0,US,107.115.20.162,unknown,Candle In The Wind - Remastered 2014,Elton John,...,NaN,NaN,trackdone,endplay,True,1.0,False,1.679089e+09,False,1L9fzw2pXprOqtemTwtxXF


In [16]:
last_six_months.to_csv('franklin_six_months.csv', index=False)

In [17]:
audio = last_six_months.drop_duplicates()

In [18]:
feature_dict = {}

# convert track_uri column to an iterable list
track_uris = audio['track_uri'].to_list()

# loop through track URIs and pull audio features using the API,
# store all these in a dictionary
for t_uri in track_uris:
    
    feature_dict[t_uri] = {'popularity': 0,
                           'danceability': 0,
                           'energy': 0,
                           'loudness':0,
                           'valence':0,
#                            'acousticness':0,
                           'speechiness': 0,
                           'instrumentalness': 0,
                           'tempo': 0}
#     audio_features_list =['artist', 'album', 'track_name', 'track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 
#                               'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'acousticness']
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    feature_dict[t_uri]['popularity'] = r['popularity']
    
    s = requests.get(BASE_URL + 'audio-features/' + t_uri, headers=headers)
    s = s.json()
    feature_dict[t_uri]['danceability'] = s['danceability']
    feature_dict[t_uri]['energy'] = s['energy']
    feature_dict[t_uri]['loudness'] = s['loudness']
    feature_dict[t_uri]['valence'] = s['valence']
#     feature_dict[t_uri]['acousticness'] = s['acousticness']
    feature_dict[t_uri]['speechiness'] = s['speechiness']
    feature_dict[t_uri]['instrumentalness'] = s['instrumentalness']
    feature_dict[t_uri]['tempo'] = s['tempo']

In [20]:
# convert dictionary into dataframe with track_uri as the first column
df_features = pd.DataFrame.from_dict(feature_dict, orient='index')
df_features.insert(0, 'track_uri', df_features.index)
df_features.reset_index(inplace=True, drop=True)

df_features

,track_uri,popularity,danceability,energy,loudness,valence,speechiness,instrumentalness,tempo
0,4FFSK1xgCLkTZJLUERLZKG,61,0.712,0.440,-11.566,0.312,0.0413,0.000000,123.958
1,5enxwA8aAbwZbf5qCHORXi,85,0.631,0.518,-8.771,0.205,0.0303,0.000000,93.023
2,0T482F5L30d7ChNbLqsKa7,43,0.414,0.235,-15.252,0.137,0.0429,0.907000,144.467
3,3EZTyh1wJEmreJH1aULPCj,62,0.323,0.106,-10.374,0.158,0.0323,0.000059,91.455
4,5Mw9bXG1dLNhbjofkVS2oR,69,0.262,0.361,-10.187,0.056,0.0333,0.002680,126.786
...,...,...,...,...,...,...,...,...,...
831,2cOUlop2ZakjOKe4WinesN,77,0.735,0.425,-6.648,0.848,0.0464,0.000000,85.896
832,0k6LV1ohcK2hlNPwu1nVw9,63,0.641,0.512,-4.935,0.395,0.0285,0.000000,116.059
833,6yhLR2sVn1IfsScVrR4ocr,60,0.416,0.341,-10.749,0.776,0.0391,0.000000,177.229
834,4pbG9SUmWIvsROVLF0zF9s,74,0.490,0.715,-5.549,0.866,0.0476,0.000000,130.726


In [23]:
df_features.to_csv('franklin_audio_features.csv', index=False)